# Learn the motion from data

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import roslib
import tf
from memmo_utils import *
import os
from geometry_msgs.msg import Quaternion, Pose, Point, Vector3
from std_msgs.msg import Header, ColorRGBA
import rospy
import os
from mlp.utils.status import Status
import mlp.utils.wholebody_result as wb_res
import matplotlib.pyplot as plt
import pickle
from IPython.display import clear_output

from transforms3d import axangles
from transforms3d import affines, quaternions

from regression import *

np.set_printoptions(precision=4, suppress=True)

%load_ext autoreload
%autoreload 2

### Start Instruction

### Load Robot Kinematics using URDF 

In [2]:
import roslib; roslib.load_manifest('urdfdom_py')
from urdf_parser_py.urdf import URDF
from pykdl_utils.kdl_parser import kdl_tree_from_urdf_model
from pykdl_utils.kdl_kinematics import KDLKinematics

robot = URDF.from_parameter_server()
tree = kdl_tree_from_urdf_model(robot)

left_foot = KDLKinematics(robot, 'base_link', 'left_sole_link')
right_foot = KDLKinematics(robot, 'base_link', 'right_sole_link')

q = left_foot.random_joint_angles()
pose = left_foot.forward(q) # forward kinematics (returns homogeneous 4x4 numpy.mat)
q_ik = left_foot.inverse(pose, q+0.3) # inverse kinematics
clear_output()

### Create Visualization

In [3]:
viz = Visual()
viz.rate = rospy.Rate(1000)

foot_marker = RvizMarker('foot_marker', 5, Marker.CUBE, 4)
colors = [ColorRGBA(0.0, 1.0, 0.0, 0.3), ColorRGBA(0.0, 0.0, 1.0, 0.3)] + [ColorRGBA(1.0, 0.0, 0.0, 0.9)]*2
foot_marker.set_color(colors)

### Option 1: Use GPR trained from the one-step movement data 

#### Load GPR 

In [ ]:
FILE_CAT = 'talos_moveEffector_flat'
#FILE_CAT = 'talos_circle'
#FILE_CAT = 'talos_circle_oriented'
ROOT_PROCESSED = '/media/teguh/Data/MEMMO Dataset/processed_data/' + FILE_CAT + '/'
ROOT_CROCS = '/media/teguh/Data/MEMMO Dataset/crocs_data/' + FILE_CAT + '/'
#based on database B
f = open(ROOT_CROCS + '/functions.pkl', 'rb')
functions = pickle.load(f)
f.close()
gpr = functions['gpr']
nn = functions['nn']
gmr = functions['gmr'] 

In [ ]:
gpr_vel = functions['gpr_vel']
gpr_u = functions['gpr_u']

In [ ]:
#based on database A
f = open(ROOT_CROCS + '/functions_A.pkl', 'rb')
functions_A = pickle.load(f)
f.close()
gpr_A = functions_A['gpr']
nn_A = functions_A['nn']
gmr_A = functions_A['gmr'] 

#### Define RBF 

In [ ]:
rbf_num = 60
timestep = 3351
Phi = define_RBF(dof=39, nbStates = rbf_num, offset = 200, width = 60, T = timestep)
plt.figure(figsize=(10,4))
plt.plot(Phi)
clear_output()

### Option 2: Train GPR from the circle movement data 

#### Loading Files (already separated into one-step movements)

In [18]:
FILE_CAT = 'talos_circle_oriented'
ROOT = '/media/teguh/Data/MEMMO Dataset/processed_data/' + FILE_CAT + '/'

f = open(ROOT + '/data_left_right.pkl', 'rb')
tic = time.time()
data1 = pickle.load(f)
f.close()
print('Data has been loaded in {} s!'.format(time.time()-tic))

Data has been loaded in 106.480104923 s!


In [22]:
keys = ['left','right']
trajs = dict()
foot_poses = dict()
x_inputs = dict()

for key in keys:
    trajs[key] = data1[key]['trajs']
    foot_poses[key] = data1[key]['foot_poses']
    x_inputs[key] = data1[key]['x_inputs']

#### Temp codes: subsample the data for Hedi

#### Visualise the dataset 

#### Subsample the data

In [20]:
len(x_inputs['left'])

1022

#### Define RBF 

In [23]:
timestep = trajs['left'].shape[1]
rbf_num = 60
Phi = define_RBF(dof=39, nbStates = rbf_num, offset = 200, width = 60, T = timestep)
#plt.plot(Phi)
clear_output()

#### Apply RBF 

In [24]:
w_trajs = dict()
for key in keys:
    w_trajs[key] = []
    for traj in trajs[key]:
        w,_,_,_ = np.linalg.lstsq(Phi, traj, rcond=0.0001)
        w_trajs[key].append(w.flatten())
    w_trajs[key] = np.array(w_trajs[key])

#### Visualize the approximated trajectories 

#### Apply PCA decomposition 

In [25]:
from sklearn.decomposition import PCA

w_trajs_pca = dict()
pca = dict()
for key in keys:
    pca[key] = PCA(n_components=60)
    w_trajs_pca[key] = pca[key].fit_transform(w_trajs[key])

#### Train GPR 

In [26]:
gpr = dict()
nn = dict()
gmr=dict()
for key in keys:
    gpr[key] = GPy_Regressor(dim_input=x_inputs[key].shape[1],is_transform=True)
    gpr[key].pca = pca[key]
    gpr[key].fit(x_inputs[key], w_trajs_pca[key],num_restarts=3)
    nn[key] = NN_Regressor()
    nn[key].fit(x_inputs[key], w_trajs_pca[key])
    nn[key].pca = pca[key]
    gmr[key] = DP_GLM_Regressor(n_components=10,n_init = 10)
    #(n_components=10,n_init=5)
    gmr[key].fit(x_inputs[key], w_trajs_pca[key])
    gmr[key].pca = pca[key]

clear_output()

#### Save the memory

In [27]:
#FILE_CAT = 'talos_moveEffector_flat'
#FILE_CAT = 'talos_circle'
FILE_CAT = 'talos_circle_oriented'
ROOT_CROCS = '/media/teguh/Data/MEMMO Dataset/crocs_data/' + FILE_CAT + '/'


functions = dict()
functions['gpr'] = gpr
functions['nn'] = nn
functions['gmr'] = gmr

f = open(ROOT_CROCS + '/functions_A_circle_oriented.pkl', 'wb')
pickle.dump(functions, f)
f.close()

#### Test Memory

In [28]:
from sklearn.mixture import GaussianMixture
gmm = dict()
for key in keys:
    gmm[key] = GaussianMixture(n_components=30, n_init = 3) 
    gmm[key].fit(x_inputs[key])

In [29]:
viz.set_rate(1000)

In [36]:
for i in range(1):
    if np.random.rand() < 0.5:
        key = 'left'
    else:
        key = 'right'

    key = 'left'
    #x = x_inputs[key][np.random.randint(x_inputs[key].shape[0])][None,:]
    x,_ = gmm[key].sample()
    func = gmr[key]#nn[key]#

    foot_marker.publish(x.reshape(-1,3))
       
    w_pca,cov = func.predict(x,return_gmm=False)
    #w_pca = w_pca + np.random.rand(1,20)*0.1
    
    #w_pca = np.random.rand(1,20)*0.4-0.2
    #w_pca = np.zeros((1,20))
    #w_pca[0,1] = -1.
    w = func.pca.inverse_transform(w_pca)
    w = w.reshape(rbf_num,-1)
    traj = np.dot(Phi,w)
    
    viz.vis_traj(traj)
    #raw_input()

In [ ]:
cov = cov[0]*10

In [ ]:
P = func.pca.components_.T
m = func.pca.mean_


In [ ]:
Phi2 = Phi[0::100]

In [ ]:
Psi = np.kron(Phi2,np.eye(39))

In [ ]:
traj_mean_dummy = (np.dot(P, w_pca.T) + m[:,None])
traj_mean = np.dot(Psi, traj_mean_dummy)

In [ ]:
a = np.dot(P,np.dot(cov,P.T))
b = np.dot(a,Psi.T)
traj_cov = np.dot(Psi,b) + 1e-7*np.eye(len(Psi))

In [ ]:
dim_in = slice(0,39)
dim_out = slice(39,975)

In [ ]:
mu_in = traj_mean[dim_in].flatten()
mu_out = traj_mean[dim_out].flatten()
sigma_in = traj_cov[dim_in,dim_in]
sigma_out = traj_cov[dim_out,dim_out]
sigma_in_out = traj_cov[dim_in,dim_out]

In [ ]:
x_in_true = traj_mean[0:39,0]

In [ ]:
x_in = x_in_true.copy()

In [ ]:
x_in[11] += 0.05

In [ ]:
mu = mu_out + np.dot(sigma_in_out.T, np.dot(np.linalg.inv(sigma_in), (x_in-mu_in).T)).flatten()
sigma = sigma_out - np.dot(sigma_in_out.T, np.dot(np.linalg.inv(sigma_in), sigma_in_out))

In [ ]:
viz.set_dof(x_in_true)

In [ ]:
viz.set_dof(x_in)

In [ ]:
x_next  = mu.reshape(24,-1)[0,:]

viz.set_dof(x_next)

In [ ]:
viz.set_rate(10)

In [ ]:
viz.vis_traj(mu.reshape(24,-1))

In [ ]:
a = slice(0,5)

In [ ]:
for i in range(10):
    traj_sample = traj_prob.rvs()
    viz.vis_traj(traj_sample.reshape(25,-1))
    raw_input()

In [ ]:
traj_mean_edit = traj_mean.reshape(250,-1)

In [ ]:
viz.set_rate(10)

In [ ]:
viz.vis_traj(traj_mean_edit)

### Predict multi-step movements using GPR at each timestep 

#### Load foot_poses examples 

In [15]:
from database import *
import os
from memmo_utils import *
from database import *
viz = Visual(rate=1000)

In [17]:
#FILE_CAT = 'talos_circle'
FILE_CAT = 'talos_circle_oriented'
#FILE_CAT = 'talos_moveEffector_flat'
ROOT = '/media/teguh/Data/MEMMO Dataset/processed_data/' + FILE_CAT + '/'

f = open(ROOT + '/raw_data.pkl', 'rb')
data2 = pickle.load(f)
foot_poses_test = data2['foot_poses']

trajs_test = data2['trajs']
data_phases_set = data2['data_phases_set']

f.close()

### Learning contact sequences 

#### Constructing the inputs

In [18]:
goal_angles = []
for i in range(len(foot_poses_test)):
    goal_angles += [foot_poses_test[i][-1,-1]]

goal_angles = np.array(goal_angles)[:,None]

#### Resample the outputs 

In [19]:
foot_pose_output = []
len_pose_output = []
num_steps = 10
for i in range(len(foot_poses_test)):
    foot_pose = foot_poses_test[i]
    len_pose_output += [len(foot_pose)]
    foot_pose = foot_pose.reshape(-1,6)
    foot_pose = subsample(foot_pose,num_steps)
    foot_pose_output += [foot_pose.flatten()]

foot_pose_output = np.array(foot_pose_output)
len_pose_output = np.array(len_pose_output)

In [20]:
from regression import *

In [21]:
gpr_pose = GPy_Regressor(dim_input = 1)

gpr_len_pose = GPy_Regressor(dim_input = 1)

In [72]:
nn_pose = NN_Regressor()
nn_len_pose = NN_Regressor()

In [23]:
gpr_pose.fit(goal_angles,foot_pose_output)

gpr_len_pose.fit(goal_angles,len_pose_output[:,None])

In [80]:
nn_pose.fit(goal_angles,foot_poses_test[:,None])
nn_len_pose.fit(goal_angles,len_pose_output[:,None])

In [83]:
foot_marker.publish(foot_pose.reshape(-1,3))

In [96]:
for foot_pose_i in (foot_pose.reshape(-1,3)):
    foot_marker.publish([foot_pose_i])
    time.sleep(1)

In [95]:
for i in range(1):
    random_angle = np.random.rand()*np.pi*2 - np.pi
    foot_pose,_ = gpr_pose.predict(np.array([[random_angle]]))
    translation = (np.cos(random_angle), np.sin(random_angle),0)
    rotation = (0,0,np.sin(0.5*random_angle), np.cos(0.5*random_angle))    
    viz.br.sendTransform(translation,rotation,rospy.Time.now(),"goal","world")
    foot_marker.publish(foot_pose.reshape(-1,3))
    #raw_input()

In [24]:
gpr_pose.save_to_file('gpr_pose')
gpr_len_pose.save_to_file('gpr_len_pose')

In [25]:
gpr_pose.load_from_file('gpr_pose')
gpr_len_pose.load_from_file('gpr_len_pose')

#### Test Prediction 

In [26]:
def calc_foot_T(x, move = 'left'):
    #calculate the footstep transformation matrix based on the input x
    #x is defined as: [left_foot, right_foot, 'foot_to_move']
    x = x.reshape(-1,3)
    Ts = []
    for x_i in x:
        T = PosetoMat(x_i)
        Ts.append(T)
    
    if move == 'left':
        T_lefts = [Ts[0], None, Ts[2]]
        T_rights = [Ts[1]]*3
    else:
        T_rights = [Ts[1], None, Ts[2]]
        T_lefts = [Ts[0]]*3
        
    return T_lefts, T_rights

In [27]:
viz.set_rate(1000)

#### Move to contact location 

In [28]:
keys = ['left', 'right']

nn = dict()
for key in keys:
    x = gpr[key].gp.X
    y = gpr[key].gp.Y
    data = np.hstack([x,y])
    nn[key] = NN_Regressor()
    nn[key].fit(x,y)

In [29]:
from ee_utils import *

In [30]:
def predict_multistep(foot_poses, func=None):
    traj_list = []
    foot_list = []
    T_lefts_list = []
    T_rights_list = []
    data_phases = []
    start_index = 0
    n = len(foot_poses)/2 - 1
    for i in range(n):
        cur_foot_pose = foot_poses[2*i:2*i+4,:].copy()
        foot_list.append(cur_foot_pose.copy())
        
        #modify the foot_pose according to the initial robot pose
        if i == 0:
            Twr = np.eye(4)
        else:
            Twr = calc_root_pose(current_q)
            Twr[2,3] = 0
            pose_wr = MattoPose(Twr,'2D')
            Twr = PosetoMat(pose_wr)
        Twr_inv = np.linalg.inv(Twr)
        
        #transform foot_pose
        for j in range(len(cur_foot_pose)):
            T = PosetoMat(cur_foot_pose[j])
            T = np.dot(Twr_inv, T)
            cur_foot_pose[j] = MattoPose(T)
            
        #predict
        key = determine_which_foot(cur_foot_pose)
        if key == 'left':
            x_input = cur_foot_pose[0:3].flatten()
        elif key == 'right':
            x_input = np.concatenate([cur_foot_pose[0:2].flatten(), cur_foot_pose[3]])
        else:
            raise 

        w_pca,cov = func[key].predict(x_input[None,:])
        w = func[key].pca.inverse_transform(w_pca)
        w = w.reshape(rbf_num,-1)
        traj = np.dot(Phi,w)
        traj = transform_traj(traj, Twr)
        current_q = traj[-1].copy()
        traj_list.append(traj)
        
        #add data_phase and contacts for crocoddyl
        #data_phases.append(np.arange(start_index + 0,start_index + 25))
        #data_phases.append(np.arange(start_index + 25,start_index + 60))
        #data_phases.append(np.arange(start_index + 60,start_index + 85))
        data_phases.append(np.arange(start_index + 0,start_index + 25))
        data_phases.append(np.arange(start_index + 25,start_index + 60))
        data_phases.append(np.arange(start_index + 60,start_index + 66))

        start_index += 66
        T_lefts, T_rights = calc_foot_T(x_input, key)
        for i in range(len(T_lefts)):
            if T_lefts[i] is not None:
                T_lefts_list += [np.dot(Twr,T_lefts[i])]
            else:
                T_lefts_list += [None]
                                 
            if T_rights[i] is not None:
                T_rights_list += [np.dot(Twr,T_rights[i])]
            else:
                T_rights_list += [None]
                
    #Cut the redundant double support phase
    num_phase = len(data_phases)
    indexes = np.arange(num_phase-3,0,-3)
    data_phases_ori = list(data_phases)

    begin_index = 0#1001
    end_index = 5#760
    traj_list[0] = traj_list[0][:-end_index,:]
    for i in range(1,len(traj_list)):
        traj_list[i] = traj_list[i][begin_index:-end_index,:]

    data_phases = list(data_phases_ori)
    for i,index in enumerate(indexes):
        del T_lefts_list[index]
        del T_rights_list[index]
        del data_phases[index]
        for j in range(index, len(data_phases)):
            data_phases[j] -= 25
                    
    return traj_list, foot_list, T_lefts_list, T_rights_list, data_phases      

In [65]:
for i in range(len(foot_pose)):
    foot_marker.publish(foot_pose[i:i+2])
    time.sleep(1)

In [67]:
for i,traj in enumerate(traj_list):
    foot_marker.publish(foot_list[i])
    raw_input()

In [31]:
def edit_foot_pose(foot_pose):
    n = len(foot_pose)/2
    for i in range(n-1):
        if i % 2 == 0:
            #left foot stationary
            foot_pose[2*i+2] = foot_pose[2*i]
        else:
            #right foot stationary
            foot_pose[2*i+3] = foot_pose[2*i+1]
    return foot_pose

In [102]:
traj_list = []
foot_list = []
T_lefts_list = []
T_rights_list = []
data_phases = []


for k in range(2,3):
    #foot_pose = foot_poses_test[np.random.randint(len(foot_poses_test))]
    #foot_pose = foot_poses_test[k]
    
    random_angle = np.random.rand()*np.pi*2 - np.pi
    #random_angle = 1.7599
    #foot_pose,_ = nn_pose.predict(np.array([[random_angle]]))    
    #foot_pose = foot_pose[0]
    #len_foot_pose,_ = nn_len_pose.predict(np.array([[random_angle]]))
    foot_pose,_ = gpr_pose.predict(np.array([[random_angle]]))
    len_foot_pose,_ = gpr_len_pose.predict(np.array([[random_angle]]))
    len_foot_pose = int(np.round(len_foot_pose[0,0]/2)*2)
    #len_foot_pose = int(np.round(len_foot_pose[0]/2)*2)
    
    print random_angle, len_foot_pose
    #foot_pose = gpr_pose.gp.Y[np.random.randint(285)]
    foot_pose = foot_pose.reshape(-1,3)
    
    #subsample
    foot_pose2 = foot_pose.reshape(-1,6)
    foot_pose2 = subsample(foot_pose2,len_foot_pose/2)
    foot_pose = foot_pose2.reshape(-1,3) 
    foot_pose = edit_foot_pose(foot_pose)
    func = gpr#nn
        

    traj_list, foot_list, T_lefts_list, T_rights_list, data_phases  = predict_multistep(foot_pose, func=func)
    print 'visualizing'
    translation = (np.cos(random_angle), np.sin(random_angle),0)
    rotation = (0,0,np.sin(0.5*random_angle), np.cos(0.5*random_angle))    
    viz.br.sendTransform(translation,rotation,rospy.Time.now(),"goal","world")

    for i,traj in enumerate(traj_list):
        foot_marker.publish(foot_list[i])
        viz.set_dof(traj[0])
        viz.set_rate(1000)
        viz.vis_traj(traj)

0.559859771933 18
visualizing


In [69]:
for i,traj in enumerate(traj_list):
    foot_marker.publish(foot_list[i])
    viz.set_dof(traj[0])
    viz.set_rate(1000)
    viz.vis_traj(traj)
    raw_input()

KeyboardInterrupt: 

In [ ]:
foot_pose = []
for f in foot_list:
    for i in f:
        foot_pose += [i]

In [ ]:
foot_marker.publish(foot_pose)

In [ ]:
for i,traj in enumerate(traj_list):
    #foot_marker.publish(foot_list[i])
    viz.set_dof(traj[0])
    viz.set_rate(1000)
    viz.vis_traj(traj)

In [ ]:
foot_marker.set_color([ColorRGBA(0.0, 1.0, 0.0, 0.8)]*foot_marker.n)

In [ ]:
FIG_FILE_NAME = "/home/teguh/git/publications/memmo_icra_2020/figures/multi_step_"


In [ ]:
viz.set_dof(traj_list[0][0])
time.sleep(0.5)
save_screenshot(150,200,750,700, FIG_FILE_NAME + str(0) + '.png') 

In [ ]:
for i,traj in enumerate(traj_list):
    viz.set_rate(1000)
    viz.vis_traj(traj)
    time.sleep(1)
    save_screenshot(150,200,750,700, FIG_FILE_NAME + str(i+1) + '.png') 

In [ ]:
sel_indexes = [-4,-3,-2,-1]

In [ ]:
def extract_root_from_traj(traj,data_phases):
    root_poses = []
    for data_phase in data_phases[2::2]:
        root_pose = calc_root_pose(traj[data_phase[-1]])
        root_poses.append(root_pose)
    return root_poses

In [ ]:
def plot_root_goals(root_poses,viz, delay = 0.6):
    for pose in root_poses:
        viz.br.sendTransform(pose[:3,3], derectify_quat(quaternions.mat2quat(pose[:3,:3])),
         rospy.Time.now(),
         "goal",
         "world"
         )
        time.sleep(delay)

#### Move to root goal

### Warmstart crocoddyl

#### Store in Crocoddyl format 

#### Reproduction data 

In [ ]:
delta_t = 0.04
traj = np.vstack(traj_list)
T_orig = traj.shape[0]

#subsample
intervals = np.concatenate([np.arange(0,T_orig, 40), [T_orig-1]])
T = data_phases[-1][-1]+1

traj = subsample(traj,T)
vel_traj = np.zeros((traj.shape[0], 38))
u_traj = np.zeros((traj.shape[0], 32))


print traj.shape, vel_traj.shape, u_traj.shape

#### From Original data 

#### Predict multi-step movements using GPR at each timestep 

In [ ]:
phases = store_for_crocoddyl(traj, vel_traj, u_traj, T_lefts_list, T_rights_list, data_phases, delta_t)
pickle.dump(phases,open('data_teguh_3.txt','wb'))

In [ ]:
from memmo_utils import *

In [ ]:
viz.set_rate(25)

examine_phases(phases, left_foot, right_foot, viz, foot_marker)

#### Run Crocoddyl 

In [ ]:
from memmo_utils import *
viz = Visual()

from crocoddyl import loadTalos
from croc_lib import *

In [ ]:
ROBOT = loadTalos('/opt/openrobots/share/example-robot-data')
problem, xs, us, ts = define_croc_problem(ROBOT,'data_teguh_3.txt', num_phases=6, is_warmstart=True)
clear_output()

In [ ]:
solver = solve_problem(ROBOT,problem,xs, us,maxiter = 50, STOP_THRESHOLD=1e-02, recalc_u=True, TYPE_OF_SOLVER='FDDP')
traj = np.array(solver.xs)[:,0:39]

In [ ]:
foot_pose = []
for f in foot_list:
    for i in f:
        foot_pose += [i]

In [ ]:
foot_marker.publish(foot_pose)

In [ ]:
viz.set_rate(40)
viz.vis_traj(traj)

### Comparison between Coldstart and Warmstart 

In [ ]:
foot_poses_test.shape

In [ ]:
from croc_lib import *
ROBOT = loadTalos('/opt/openrobots/share/example-robot-data')


In [ ]:
viz.set_rate(1000)

#### For video

In [ ]:
gpr_trajs, croc_trajs = [],[]
foot_steps = []

In [ ]:
foot_marker.publish(foot_poses_test[29])

In [ ]:
keys = ['left','right']
file_name = 'data.txt'
 
delta_t = 0.04
result = dict()
res_keys = ['gpr', 'cold']
wm_methods = [gpr]
for res_key in res_keys:
    result[res_key] = dict()
    result[res_key]['trajs'] = []
    result[res_key]['vel_trajs'] = []
    result[res_key]['u_trajs'] = []
    
    result[res_key]['n_iters'] = []
    result[res_key]['costs'] = []
    result[res_key]['foot_pose'] = []

trajs_true = []
    
#indexes = [2,12,32,43,54]
indexes = [29]
#for i in range(35,100):
for i in indexes[:]:
    #for i in range(len(x_inputs_test[key])):
    print 'Iteration ' + key + str(i)
    #create problem
    foot_pose = foot_poses_test[i]
    foot_steps += [foot_pose]
    foot_marker.publish(foot_pose)
    traj_true = trajs_test[i]
    trajs_true.append(traj_true)

    #viz.set_rate(1000)
    #viz.vis_traj(traj_true)
    q_init = traj_true[0]
    v_init = np.zeros(38)


    #warmstart method
    for j, func in enumerate(wm_methods):
        res_key = res_keys[j]
        #predict the trajectory
        #func = gpr#nn
        traj_list, foot_list, T_lefts_list, T_rights_list, data_phases  = predict_multistep(foot_pose, func=func)

        traj = np.vstack(traj_list)
        T_orig = traj.shape[0]
        #subsample
        intervals = np.concatenate([np.arange(0,T_orig, 40), [T_orig-1]])
        T = data_phases[-1][-1]+1

        traj = subsample(traj,T)
        gpr_trajs.append(traj)
        viz.set_rate(40)
        #viz.vis_traj(traj)
        vel_traj = np.zeros((traj.shape[0], 38))
        u_traj = np.zeros((traj.shape[0], 32))


        phases = store_for_crocoddyl(traj, vel_traj, u_traj, T_lefts_list, T_rights_list, data_phases, delta_t, q_init=q_init, v_init=v_init)
        file_name='data_teguh_3.txt'
        pickle.dump(phases,open(file_name,'wb'))

        print 'warmstart step' + res_key
        problem, xs, us, ts = define_croc_problem(ROBOT,file_name, num_phases=len(phases), is_warmstart=True)

        solver = solve_problem(ROBOT,problem,xs, us,maxiter = 20, STOP_THRESHOLD=1e-02, recalc_u=True, TYPE_OF_SOLVER='FDDP')
        traj = np.array(solver.xs)[:,0:39]
        croc_trajs.append(traj)
        viz.set_rate(40)

        #viz.vis_traj(traj)
        #raw_input()
    clear_output()

In [ ]:
keys = ['left','right']
file_name = 'data.txt'
 
delta_t = 0.04
result = dict()
res_keys = ['gpr', 'gmr', 'gpr_A', 'gmr_A','cold', 'q', 'qu', 'qv', 'qvu','q_nou', 'u']
#wm_methods = [gpr,gmr,gpr_A, gmr_A]
#wm_methods = [gpr,gmr]
#wm_methods = [gpr]
for res_key in res_keys:
    result[res_key] = dict()
    result[res_key]['trajs'] = []
    result[res_key]['vel_trajs'] = []
    result[res_key]['u_trajs'] = []
    
    result[res_key]['n_iters'] = []
    result[res_key]['costs'] = []
    result[res_key]['foot_pose'] = []

trajs_true = []
    

for i in range(35):
    for key in keys:
        #for i in range(len(x_inputs_test[key])):
        print 'Iteration ' + key + str(i)
        #create problem
        foot_pose = foot_poses_test[i]
        foot_marker.publish(foot_pose)
        traj_true = trajs_test[i]
        trajs_true.append(traj_true)

        #viz.set_rate(1000)
        #viz.vis_traj(traj_true)
        q_init = traj_true[0]
        v_init = np.zeros(38)


        #warmstart method
        for j, func in enumerate(wm_methods):
            res_key = res_keys[j]
            #predict the trajectory
            #func = gpr#nn
            traj_list, foot_list, T_lefts_list, T_rights_list, data_phases  = predict_multistep(foot_pose, func=func)

            traj = np.vstack(traj_list)
            T_orig = traj.shape[0]
            #subsample
            intervals = np.concatenate([np.arange(0,T_orig, 40), [T_orig-1]])
            T = data_phases[-1][-1]+1

            traj = subsample(traj,T)

            viz.set_rate(40)
            viz.vis_traj(traj)
            vel_traj = np.zeros((traj.shape[0], 38))
            u_traj = np.zeros((traj.shape[0], 32))


            phases = store_for_crocoddyl(traj, vel_traj, u_traj, T_lefts_list, T_rights_list, data_phases, delta_t, q_init=q_init, v_init=v_init)
            file_name='data_teguh_3.txt'
            pickle.dump(phases,open(file_name,'wb'))

            print 'warmstart step' + res_key
            problem, xs, us, ts = define_croc_problem(ROBOT,file_name, num_phases=7, is_warmstart=True)

            solver = solve_problem(ROBOT,problem,xs, us,maxiter = 20, STOP_THRESHOLD=1e-02, recalc_u=True, TYPE_OF_SOLVER='FDDP')
            traj = np.array(solver.xs)[:,0:39]
            viz.set_rate(40)

            #viz.vis_traj(traj)
            result[res_key]['trajs'] += [traj]
            result[res_key]['u_trajs'] = [solver.us]
            result[res_key]['n_iters'] += [solver.iter]
            result[res_key]['costs'] += [solver.cost]
            result[res_key]['foot_pose'] += [foot_pose]

        print 'coldstart step'
        problem, xs, us, ts = define_croc_problem(ROBOT,file_name, num_phases=7, is_warmstart=False)

        solver = solve_problem(ROBOT,problem,xs, us,maxiter = 20, STOP_THRESHOLD=1e-02, recalc_u=True, TYPE_OF_SOLVER='FDDP')
        traj = np.array(solver.xs)[:,0:39]
        res_key = 'cold'
        result[res_key]['trajs'] += [traj]
        result[res_key]['u_trajs'] = [solver.us]
        result[res_key]['n_iters'] += [solver.iter]
        result[res_key]['costs'] += [solver.cost]
        result[res_key]['foot_pose'] += [foot_pose]
        clear_output()

        success = dict()
        for res_key in res_keys:
            success[res_key] = np.array(result[res_key]['costs']) < 1e3
            print"{0} \t & {1:.2f} & \t {2:.2f} $\\pm${3:.2f} & \t {4:.2f} $\\pm$ {5:.2f}".format(res_key,  100.*np.sum(success[res_key])/(len(success[res_key])+.000001), np.mean(np.array(result[res_key]['costs'])[success[res_key]]), np.std(np.array(result[res_key]['costs'])[success[res_key]]), np.mean(np.array(result[res_key]['n_iters'])[success[res_key]]),np.std(np.array(result[res_key]['n_iters'])[success[res_key]]))

        if i%20 == 0:
            f = open(ROOT_CROCS + '/result_multistep_circle.pkl', 'wb')
            pickle.dump(result,f)
            f.close()

In [ ]:
sel_indexes = [-4,-3,-2,-1]

In [ ]:
sel_indexes = [0,1,2,4]

In [ ]:
foot_marker.publish(foot_steps[0])

In [ ]:
for i in sel_indexes[-1:]:
    print i
    foot_marker.publish(foot_steps[i])
    viz.vis_traj(gpr_trajs[i])
    raw_input()

In [ ]:
FIG_FILE_NAME = "/home/teguh/git/publications/memmo_icra_2020/videos/multistep_gpr_"

In [ ]:
record_video(np.array(foot_steps)[sel_indexes],np.array(gpr_trajs)[sel_indexes], FIG_FILE_NAME)

In [ ]:
os.system('ffmpeg -r 25 -start_number 0 -i /home/teguh/git/publications/memmo_icra_2020/videos/multistep_gpr_%d.png -c:v libx264 -r 30 -pix_fmt yuv420p /home/teguh/git/publications/memmo_icra_2020/videos/multistep_gpr.mp4')

In [ ]:
FIG_FILE_NAME = "/home/teguh/git/publications/memmo_icra_2020/videos/multistep_croc_"

In [ ]:
record_video(np.array(foot_steps)[sel_indexes],np.array(croc_trajs)[sel_indexes], FIG_FILE_NAME)

In [ ]:
os.system('ffmpeg -r 25 -start_number 0 -i /home/teguh/git/publications/memmo_icra_2020/videos/multistep_croc_%d.png -c:v libx264 -r 30 -pix_fmt yuv420p /home/teguh/git/publications/memmo_icra_2020/videos/multistep_croc.mp4')

In [ ]:
filename = '/home/teguh/git/publications/memmo_icra_2020/videos/final_video_'

In [ ]:
#time.sleep(5)
tic = time.time()
save_screenshot(pb,240,0,1440,900, filename + '00.png')
toc = time.time()
print toc-tic

In [ ]:
import gtk.gdk

def save_screenshot(x,y,w,h,file_name):
    window = gtk.gdk.get_default_root_window()
    pb = gtk.gdk.Pixbuf(gtk.gdk.COLORSPACE_RGB,False,8,w,h)
    pb = pb.get_from_drawable(window,window.get_colormap(),x,y,0,0,w,h)
    #pb = pb.subpixbuf(int(x),int(y),int(w),int(h)) 
    if (pb != None):
        pb.save(file_name,"png")
        #print "Screenshot saved."

In [ ]:
import gtk.gdk
def save_screenshot(x,y,w,h,file_name):
    window = gtk.gdk.get_default_root_window()
    sz = window.get_size()
    print "The size of the window is %d x %d" % sz
    pb = gtk.gdk.Pixbuf(gtk.gdk.COLORSPACE_RGB,False,8,sz[0],sz[1])
    pb = pb.get_from_drawable(window,window.get_colormap(),0,0,0,0,sz[0],sz[1])
    pb = pb.subpixbuf(int(x),int(y),int(w),int(h)) 
    if (pb != None):
        pb.save(file_name,"png")
        print "Screenshot saved."
    else:
        print "Unable to get the screenshot."

In [ ]:
time.sleep(5)
#for i in range(3600):
for i in range(600):
    print i
    save_screenshot(240,0,1440,900, filename + str(i) + '.png')
    #time.sleep(0.0333)

In [ ]:
os.system('ffmpeg -r 10 -start_number 0 -i /home/teguh/git/publications/memmo_icra_2020/videos/final_video_%d.png -c:v libx264 -r 30 -pix_fmt yuv420p /home/teguh/git/publications/memmo_icra_2020/videos/final_video.mp4')

In [ ]:
def record_video(foot_steps, trajs,file_name):
    fig_index = 0
    for i,traj in enumerate(trajs):
        foot_marker.publish([])
        foot_marker.publish(foot_steps[i])
        viz.set_dof(traj[0])
        time.sleep(0.2)
        for j,index in enumerate(np.arange(0,len(traj))):
            viz.set_dof(traj[index])
            time.sleep(0.02)
            save_screenshot(150,200,800,700, file_name +  str(fig_index) + '.png') 
            fig_index +=1
        for k in range(20):
            save_screenshot(150,200,800,700, file_name  +  str(fig_index)  + '.png') 
            fig_index +=1
        clear_output()

In [ ]:
f = open(ROOT_CROCS + '/result_multistep_circle.pkl', 'wb')
pickle.dump(result,f)
f.close()

In [ ]:
f = open(ROOT_CROCS + '/result_multistep2.pkl', 'wb')
pickle.dump(result,f)
f.close()

In [ ]:
f = open(ROOT_CROCS + '/result_multistep.pkl', 'rb')
result2 = pickle.load(f)
f.close()

In [ ]:
success = dict()
for res_key in res_keys:
    success[res_key] = np.array(result2[res_key]['costs']) < 1e3
    print"{0} \t & {1:.2f} & \t {2:.2f} $\\pm${3:.2f} & \t {4:.2f} $\\pm$ {5:.2f}".format(res_key,  100.*np.sum(success[res_key])/(len(success[res_key])+.000001), np.mean(np.array(result2[res_key]['costs'])[success[res_key]]), np.std(np.array(result2[res_key]['costs'])[success[res_key]]), np.mean(np.array(result2[res_key]['n_iters'])[success[res_key]]+1),np.std(np.array(result2[res_key]['n_iters'])[success[res_key]]+1))

In [ ]:
success = dict()
for res_key in res_keys:
    success[res_key] = np.array(result[res_key]['costs']) < 1e3
    print"{0} \t & {1:.2f} & \t {2:.2f} $\\pm${3:.2f} & \t {4:.2f} $\\pm$ {5:.2f}".format(res_key,  100.*np.sum(success[res_key])/(len(success[res_key])+.000001), np.mean(np.array(result[res_key]['costs'])[success[res_key]]), np.std(np.array(result[res_key]['costs'])[success[res_key]]), np.mean(np.array(result[res_key]['n_iters'])[success[res_key]]+1),np.std(np.array(result[res_key]['n_iters'])[success[res_key]]+1))

In [ ]:
gpr_A[key].gp.Y.shape

In [ ]:
len(solver.problem.runningDatas)

In [ ]:
a = solver.problem.runningDatas[3]

In [ ]:
 a.differential.costs.costs

In [ ]:
b = a.differential.costs.costs['ctrlReg']

In [ ]:
b.cost

In [ ]:
b.cost

In [ ]:
viz.vis_traj(result['gpr']['trajs'][0])

In [ ]:
viz.vis_traj(result['gpr_A']['trajs'][0])

In [ ]:
viz.vis_traj(result['gmr']['trajs'][0])

In [ ]:
viz.vis_traj(result['cold']['trajs'][0])

In [ ]:
index = 0
traj1 = result['gpr']['trajs'][index]
traj2 = result['cold']['trajs'][index]

In [ ]:
for i in range(traj1.shape[1]):
    print i
    plt.plot(traj1[:,i],'r')
    plt.plot(traj2[:,i],'b')
    plt.show()
    raw_input()
    plt.clf()
    clear_output()

In [ ]:
    
    #predict the trajectories
    w_pca,cov = gpr[key].predict(x_input)
    w = gpr[key].pca.inverse_transform(w_pca)
    w = w.reshape(rbf_num,-1)
    traj = np.dot(Phi,w)

    w_pca,cov = gpr_vel[key].predict(x_input)
    w = gpr_vel[key].pca.inverse_transform(w_pca)
    w = w.reshape(rbf_num,-1)
    vel_traj = np.dot(Phi,w)

    w_pca,cov = gpr_u[key].predict(x_input)
    w = gpr_u[key].pca.inverse_transform(w_pca)
    w = w.reshape(rbf_num,-1)
    u_traj = np.dot(Phi,w)

    wm_comps = ['q', 'qu', 'q_nou', 'u']
    for res_key in wm_comps:
        if res_key == 'q':
            inputs = [traj.copy(),None,None]
            recalc = True
        elif res_key == 'q_nou':
            inputs = [traj.copy(),None,None]
            recalc = False
        elif res_key == 'qu':
            inputs = [traj.copy(),None,u_traj]
            recalc = False
        elif res_key == 'qv':
            inputs = [traj.copy(),vel_traj,None]
            recalc = True
        elif res_key == 'qvu':
            inputs = [traj.copy(),vel_traj,u_traj]
            recalc = False
        elif res_key == 'u':
            inputs = [traj.copy()*0,None,u_traj]
            recalc = False
            
        
        traj_mod,_ = convert_to_croc_format(x_input,traj=inputs[0], vel_traj = inputs[1], \
                                u_traj = inputs[2], key = key, data_phases = data_phases_ori,\
                                q_init = q_init, v_init = v_init, filename=file_name)
        print 'warmstart step' + res_key
        problem, xs, us, ts = define_croc_problem(ROBOT,file_name, num_phases=3, is_warmstart=True)

        solver = solve_problem(ROBOT,problem,xs, us,maxiter = 20, STOP_THRESHOLD=1e-02, recalc_u=recalc, TYPE_OF_SOLVER='FDDP')
        res_traj = np.array(solver.xs)[:,0:39]
        result[res_key]['trajs'] += [res_traj]
        result[res_key]['vel_trajs'] += [np.array(solver.xs)[:,39:]]
        result[res_key]['u_trajs'] += [np.array(solver.us)]
        result[res_key]['n_iters'] += [solver.iter]
        result[res_key]['costs'] += [solver.cost]
        result[res_key]['x_inputs'] += [x_input]
    
    
    
    
    clear_output()